In [62]:
import json
import os
from typing import List
import networkx as nx
import nltk
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
from annotated_text import annotated_text, parameters
from streamlit_extras import add_vertical_space as avs
from streamlit_extras.badges import badge
from scripts.similarity import get_similarity_score, find_path, read_config
from scripts.utils import get_filenames_from_dir
from scripts import ReadPdf, JobDescriptionProcessor, ResumeProcessor, KeytermsExtraction
import cohere
from scripts.KeytermsExtraction import KeytermExtractor
from scripts.similarity.get_similarity_score import get_similarity_score
import uuid
from langchain.embeddings import HuggingFaceEmbeddings

In [63]:
import yaml
import logging
logging.basicConfig(
    filename='app_similarity_score.log',
    filemode='w',
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

console_handler = logging.StreamHandler()
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s")
console_handler.setFormatter(formatter)
console_handler.setLevel(logging.DEBUG)

file_handler = logging.FileHandler("app_similarity_score.log")
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(console_handler)
cwd = find_path('Resume-Matcher')
config_path = os.path.join(cwd, "scripts", "similarity")


def read_config(filepath):
    try:
        with open(filepath) as f:
            config = yaml.safe_load(f)
        return config
    except FileNotFoundError as e:
        logger.error(f"Configuration file {filepath} not found: {e}")
    except yaml.YAMLError as e:
        logger.error(
            f"Error parsing YAML in configuration file {filepath}: {e}", exc_info=True)
    except Exception as e:
        logger.error(f"Error reading configuration file {filepath}: {e}")
    return None


config = read_config(config_path + "/config.yml")
PROJECT_ID = config['vertex']['api_key']
REGION = config['vertex']['REGION']

In [64]:
FULL_STACK = 0

In [65]:
import os

job_desc_directory = "Data/JobDescription/"
resumes_directory = "Data/Resumes/"

# Automatically get all job description and resume files
job_desc_files = [file for file in os.listdir(
    job_desc_directory) if file.endswith('.pdf')]
resume_files = [file for file in os.listdir(
    resumes_directory) if file.endswith('.pdf')]

# This will select the first job description file
job_desc_file = job_desc_files[FULL_STACK]

## Connect your MongoDB database

Pinged your deployment. You successfully connected to MongoDB!


In [6]:
# Choose or create a database named 'resume_db'
db = client['resume_db']

# Choose or create a collection named 'candidates'
candidates_collection = db['candidates']

In [7]:
# Choose or create a database named 'job_db'
db = client['job_db']
job_collection = db['job']

In [8]:
# Define paths
# Read raw job description
# job_desc_text = ReadPdf.read_single_pdf("Data/JobDescription/" + job_desc_file)
job_desc_text = [ReadPdf.read_single_pdf(os.path.join(
    "Data/JobDescription/", job_desc_file)) for job_desc_file in job_desc_files]
# Process job description
job_desc_processor = JobDescriptionProcessor(job_desc_file)
job_desc_processed = job_desc_processor._read_job_desc()

job_files = [f for f in os.listdir(
    "Data/JobDescription/") if os.path.isfile(os.path.join("Data/JobDescription/", f))]


job_processed = []
for job_file in job_files:
    job_processor = JobDescriptionProcessor(job_file)
    job_data = job_processor._read_job_desc()
    job_processor._write_json_file(job_data)
    job_processed.append(job_data)

In [9]:

resumes_text = [ReadPdf.read_single_pdf(os.path.join(
    "Data/Resumes/", resume_file)) for resume_file in resume_files]


# Process resumes
resume_files = [f for f in os.listdir(
    "Data/Resumes/") if os.path.isfile(os.path.join("Data/Resumes/", f))]

resumes_processed = []
for resume_file in resume_files:
    resume_processor = ResumeProcessor(resume_file)
    resume_data = resume_processor._read_resumes()
    resume_processor._write_json_file(resume_data)
    resumes_processed.append(resume_data)

In [10]:
def extract_candidate_name_from_filename(filename: str) -> str:
    # Exclude the last part which is the position
    name_parts = filename.split('_')[:-1]
    return ' '.join(name_parts).title()

In [11]:
candidate_names = [extract_candidate_name_from_filename(
    resume_file) for resume_file in resume_files]
candidate_names

['Yiching Liu',
 'Angela Zhu',
 'Maria Chinkan',
 'Arpi Melik Parsadanyan',
 'Zihui Lin',
 'Jose Felix Villasenor',
 'Anna Gasparyan',
 'Amitesh Rathore',
 'Yunrui Shao',
 'Jaykumar',
 'Minyue Yao',
 'John',
 'Zane Rouguine',
 'Brandon Penner',
 'Zhe Wang',
 'Tsubasa Lin',
 'Danny Mai',
 'David Boutwell',
 'Alexandra',
 'Sarah Sherman',
 'David Botbol',
 'Ryan Pintar',
 'Eloise Yu',
 'Vasil Klimovich',
 'Ming Jin',
 'Robert Scozzari',
 'Cody Romero',
 'Carnell Brame',
 'Timothy Wang',
 'Nico Santoso',
 'Shirley Zhao',
 'Yuan Wang',
 'Divya Harshini',
 'Deekshitha Pullaiah',
 'Meredith Cheng',
 'Grace Li',
 'Anya Hsu',
 'Nandini Seth',
 'Andrew Knuppel',
 'John Hinnegan',
 'Sharad Dangol',
 'Salvador Campos',
 'Lauren Aubrey Lee',
 'Xiao Li',
 'Bruce Wayne',
 'Barry Allen',
 'Balraj Rai',
 'Annie Zhou',
 'Federico De Marines',
 'Ray Lee',
 'Jagriti Sharma',
 'Michelle Wang',
 'Galen Fink',
 'Dennis Mo',
 'Mengyao Zhang',
 'Johann C',
 'Yixin-Ying',
 'Peggy Lai',
 'Yuanhuang Lo',
 'Serle

In [12]:
# resumes_processed[0]['name']

for i in range(len(resumes_processed)):
    resumes_processed[i]['name'] = candidate_names[i]

In [13]:
candidates_collection.insert_many(resumes_processed)

In [14]:
def extract_jobdes_name_from_filename(filename: str) -> str:
    # Exclude the last part which is the position
    name_parts = filename.split('_')[:-1]
    return ' '.join(name_parts).title()

In [15]:
job_des = [extract_jobdes_name_from_filename(
    job_desc_file) for job_desc_file in job_desc_files]
job_des = [item.replace('Job Desc ', '') for item in job_des]

In [16]:
for i in range(len(job_processed)):
    job_processed[i].update({'job_title': job_des[i]})

In [17]:
job_collection.insert_many(job_processed)

In [18]:
job_des_parse_data = job_collection.find({}, {'unique_id':1,'clean_data':1,'extracted_keywords':1,'_id':0,'job_title':1})

In [19]:
job_des_parse_data = list(job_des_parse_data)

In [20]:
job_title_to_index = {}
job_unique_id = []

for index, data in enumerate(job_des_parse_data):
    # Convert the job_title to uppercase and replace spaces with underscores
    constant_name = data['job_title'].upper().replace(' ', '_')
    job_unique_id.append(data['unique_id'])
    # Set the constant name as a key in the dictionary with its index as the value
    job_title_to_index[constant_name] = index

# To access a particular index:
# index_for_full_stack = job_title_to_index['FULL_STACK']

In [21]:
# Get all the candidates from the MongoDB collection
candidates_parse_data = candidates_collection.find(
    {}, {"name": 1, "unique_id": 1, "_id": 0, "clean_data": 1, "extracted_keywords": 1})

In [22]:
candidates_parse_data = list(candidates_parse_data)

In [23]:
candidates_parse_data_holder = candidates_parse_data

In [24]:
candidates_unique_id = []

for index, data in enumerate(candidates_parse_data):
    # Convert the job_title to uppercase and replace spaces with underscores
    candidates_unique_id.append(data['unique_id'])
    # Set the constant name as a key in the dictionary with its index as the value

In [25]:
# Initialize an empty list for resumes_keywords if it doesn't exist yet


# Iterate over the cursor and update resumes_keywords
# Initialize an empty list for resumes_keywords if it doesn't exist yet
# Initialize an empty list for resumes_keywords if it doesn't exist yet
resumes_keywords_str_list = []

# Iterate over the cursor and update resumes_keywords
for document in candidates_parse_data_holder:
    keyword_string = ' '.join(document['extracted_keywords'])
    resumes_keywords_str_list.append(keyword_string)
    # resumes_keywords.append(document['extracted_keywords'])


# for index in range(len(candidates_parse_data)):
#     candidates_parse_data[index]['extracted_keywords'] = resumes_keywords[index]

In [26]:
resumes_clean_data_str_list = []

# Iterate over the cursor and update resumes_keywords
for document in candidates_parse_data:
    keyword_string = ' '.join(document['clean_data'])
    resumes_clean_data_str_list.append(keyword_string)

In [27]:
# Convert the extracted keywords into strings
# job_desc_keywords_str = ' '.join([keyword[0] for keyword in job_desc_keywords])
job_desc_keywords_str_list = []
for index in range(len(job_des_parse_data)):
    keyword_string = ' '.join(document['extracted_keywords'])
    job_desc_keywords_str_list.append(keyword_string)
    #job_desc_keywords_str_list.append(job_des_parse_data[index]['clean_data'])
    #job_desc_keywords_str_list.append(job_des_parse_data[index]['extracted_keywords'])
    

In [28]:
candidate_names = [extract_candidate_name_from_filename(
    resume_file) for resume_file in resume_files]

In [29]:
embeddings = HuggingFaceEmbeddings()

/Users/liuchuning/USC/Intellipro/Resume-Matcher/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
#job_desc_embeddings = embeddings.embed_documents(job_desc_keywords_str_list)
# Assuming job_desc_keywords_str_list is a list of lists
# flattened_list = [
#     item for sublist in job_desc_keywords_str_list for item in sublist]

# Now, each element of flattened_list should be a string
job_desc_embeddings = embeddings.embed_documents(job_desc_keywords_str_list)

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


In [31]:
resumes_embeddings = embeddings.embed_documents(resumes_keywords_str_list)

Batches: 100%|██████████| 15/15 [00:25<00:00,  1.70s/it]


## Connect your Pinecone database

In [32]:
import pinecone

pinecone.init(
	api_key= config['pinecone']['api_key'],
	environment='gcp-starter'
)

index = pinecone.Index(index_name="jobmatcher")

In [33]:
index_for_full_stack = job_title_to_index['FULL_STACK']

In [34]:
job_des_underscore = [name.replace(' ', '_') for name in job_des]

In [35]:
candidate_names_underscore = [name.replace(
    ' ', '_') for name in candidate_names]

In [36]:
print(len(job_desc_embeddings), len(job_des_underscore), len(job_unique_id))

20 4 20


In [42]:
job_vectors_with_metadata = []

# for i, embedding in enumerate(job_desc_embeddings):
#     vector_data = {
#         "id": job_des_underscore[i:4],
#         "values": embedding,
#         "metadata": {"unique_id": job_unique_id[i]}
#     }
#     job_vectors_with_metadata.append(vector_data)

In [43]:
for i, embedding in enumerate(job_desc_embeddings):
    if i < len(job_des_underscore) and i < len(job_unique_id):
        vector_data = {
            "id": job_des_underscore[i],
            "values": embedding,
            "metadata": {"unique_id": job_unique_id[i]}
        }
        job_vectors_with_metadata.append(vector_data)
    else:
        break  # Break the loop if i exceeds the length of other lists

In [44]:
job_vectors_with_metadata

[{'id': 'Full_Stack',
  'values': [0.06383941322565079,
   0.04743417724967003,
   -0.04527309164404869,
   -0.019894350320100784,
   0.0627434253692627,
   0.015805304050445557,
   0.04794888570904732,
   7.700078276684508e-05,
   0.014157479628920555,
   -0.0025394069962203503,
   0.03456815332174301,
   0.040812090039253235,
   0.004812612663954496,
   0.09908850491046906,
   -0.008227513171732426,
   -0.018342191353440285,
   0.02843921072781086,
   -0.001228014356456697,
   -0.06582285463809967,
   0.000989047810435295,
   -0.026163538917899132,
   0.0835939273238182,
   -0.03525662049651146,
   0.030543969944119453,
   0.06994826346635818,
   0.004977548494935036,
   0.029003744944930077,
   0.023978684097528458,
   0.003741516964510083,
   -0.02136107347905636,
   0.04859698936343193,
   -0.02851484715938568,
   -0.04444178566336632,
   0.025382090359926224,
   2.3381548999168444e-06,
   -0.013515641912817955,
   -0.011791647411882877,
   -0.007770450320094824,
   -0.03582251071

In [49]:
candidates_vectors_with_metadata = []

# for i, embedding in enumerate(resumes_embeddings):
#     vector_data = {
#        "id": candidate_names_underscore[i:4],
#         "values": embedding,
#         "metadata": {"unique_id": candidates_unique_id[i]}
#     }
#     candidates_vectors_with_metadata.append(vector_data)
for i, embedding in enumerate(resumes_embeddings):
    if i < len(candidate_names_underscore) and i < len(candidates_unique_id):
        vector_data = {
            "id": candidate_names_underscore[i],
            "values": embedding,
            "metadata": {"unique_id": candidates_unique_id[i]}
        }
        candidates_vectors_with_metadata.append(vector_data)
    else:
        break  # Break the loop if i exceeds the length of other lists

In [50]:
all_vectors_with_metadata = job_vectors_with_metadata + candidates_vectors_with_metadata
index.upsert(vectors=all_vectors_with_metadata)

{'upserted_count': 97}

In [51]:
candidates_fetched_vectors = index.fetch(ids=candidate_names_underscore)

In [52]:
jobs_fetched_vectors = index.fetch(ids=job_des_underscore)

In [53]:
all_job_vectors = {}

for name, data in jobs_fetched_vectors['vectors'].items():
    all_job_vectors[name] = data['values']

In [55]:
all_candidate_vectors = {}

for name, data in candidates_fetched_vectors['vectors'].items():
    all_candidate_vectors[name] = data['values']

In [56]:
all_job_vectors['Full_Stack']

[0.0638394132,
 0.0474341772,
 -0.0452730916,
 -0.0198943503,
 0.0627434254,
 0.0158053041,
 0.0479488857,
 7.70007828e-05,
 0.0141574796,
 -0.002539407,
 0.0345681533,
 0.04081209,
 0.00481261266,
 0.0990885049,
 -0.00822751317,
 -0.0183421914,
 0.0284392107,
 -0.00122801436,
 -0.0658228546,
 0.00098904781,
 -0.0261635389,
 0.0835939273,
 -0.0352566205,
 0.0305439699,
 0.0699482635,
 0.00497754849,
 0.0290037449,
 0.0239786841,
 0.00374151696,
 -0.0213610735,
 0.0485969894,
 -0.0285148472,
 -0.0444417857,
 0.0253820904,
 2.3381549e-06,
 -0.0135156419,
 -0.0117916474,
 -0.00777045032,
 -0.0358225107,
 -0.0181787871,
 0.0226532798,
 0.0118414089,
 -0.00581042422,
 0.0203245636,
 -0.00688380562,
 -0.0174786411,
 0.0536851436,
 0.00832477398,
 -0.0178664047,
 0.0289549939,
 0.0214646924,
 0.0125116818,
 -0.033652015,
 -0.0508549027,
 0.00525621,
 0.0289512556,
 -0.0248817466,
 0.0365113914,
 0.0593514442,
 -0.00184022845,
 0.0192789193,
 -0.0276137181,
 -0.0333505943,
 -0.042342104,
 0.07

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_cosine_similarity(embedding1: List[float], embedding2: List[float]) -> float:
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    return similarity

In [59]:

similarities_with_names = []

# Assuming all_job_vectors['Full_Stack'] gives you the vector for the 'Full_Stack' job.
full_stack_vector = all_job_vectors['Full_Stack']

for candidate_name, candidate_vector in all_candidate_vectors.items():
    similarity_score = cosine_similarity(
        [full_stack_vector], [candidate_vector])[0][0]
    similarities_with_names.append({
        "name": candidate_name,
        "similarity": similarity_score
    })

In [60]:
ranked_candidates = sorted(similarities_with_names,
                           key=lambda x: x["similarity"], reverse=True)

# Print the ranked candidates
# Starting the index from 1
for index, candidate in enumerate(ranked_candidates, 1):
    print(
        f"Candidate {index}: {candidate['name']}, Similarity Score: {candidate['similarity']:.4f}")

Candidate 1: Polina_Popova, Similarity Score: 1.0000
Candidate 2: Chenjie_Wu, Similarity Score: 0.8689
Candidate 3: Nico_Santoso, Similarity Score: 0.8631
Candidate 4: Adrian_Velasco, Similarity Score: 0.8535
Candidate 5: Serleen_Lee, Similarity Score: 0.8490
Candidate 6: Rulin_Xing, Similarity Score: 0.8418
Candidate 7: Divya_Harshini, Similarity Score: 0.8408
Candidate 8: Charles_Zhang, Similarity Score: 0.8323
Candidate 9: Anya_Hsu, Similarity Score: 0.8236
Candidate 10: Eloise_Yu, Similarity Score: 0.8200
Candidate 11: Hanfei_He, Similarity Score: 0.8199
Candidate 12: Yunrui_Shao, Similarity Score: 0.8185
Candidate 13: Annie_Zhou, Similarity Score: 0.8137
Candidate 14: Yihao_Xu, Similarity Score: 0.8134
Candidate 15: Jeffrey_Chen, Similarity Score: 0.7967
Candidate 16: Michelle_Wang, Similarity Score: 0.7921
Candidate 17: Zihui_Lin, Similarity Score: 0.7901
Candidate 18: Arif_Demirkan, Similarity Score: 0.7893
Candidate 19: Grace_Li, Similarity Score: 0.7824
Candidate 20: Dennis_To

In [61]:
import csv

# Set the file path
file_path = 'Data/Result/ranked_candidates_hugging.csv'

# Prepare the data for CSV
rows = [["Rank", "Candidate Name", "Similarity Score"]]
for index, candidate in enumerate(ranked_candidates, 1):
    rows.append([index, candidate['name'], candidate['similarity']])

# Save to CSV
with open(file_path, "w", newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)